In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [4]:
data = pd.read_csv('/gdrive/MyDrive/Colab Notebooks/ML Project/AVAX-USDT_30m_22-09-2020_09-12-2022.csv')
data.describe

<bound method NDFrame.describe of                datetime_id     open     high      low    close      volume
0      2020-09-22 09:30:00   0.8500   6.0000   0.8500   4.8811  3198372.67
1      2020-09-22 10:00:00   4.8800   5.3500   4.2450   4.9312  1914636.29
2      2020-09-22 10:30:00   4.9141   5.1999   4.5662   4.9096  1256377.31
3      2020-09-22 11:00:00   4.9096   5.4499   4.8200   5.3951  1100528.85
4      2020-09-22 11:30:00   5.3917   6.9289   5.3373   6.8219  2783452.58
...                    ...      ...      ...      ...      ...         ...
38755  2022-12-09 15:30:00  13.5900  13.5900  13.4000  13.4700    80416.54
38756  2022-12-09 16:00:00  13.4600  13.4800  13.3600  13.4100    21615.24
38757  2022-12-09 16:30:00  13.4200  13.5100  13.3800  13.4900    19410.28
38758  2022-12-09 17:00:00  13.5000  13.5400  13.4500  13.5000    31878.00
38759  2022-12-09 17:30:00  13.5000  13.5300  13.5000  13.5100     4308.78

[38760 rows x 6 columns]>

In [5]:
def window_input(window_length: int, data: pd.DataFrame) -> pd.DataFrame:
    
    df = data.copy()
    
    i = 1
    while i < window_length:
        for col in ['open', 'high', 'low', 'close', 'volume']:
            df[f'{col}_{i}'] = df[col].shift(-i)
        i = i + 1
        
    if i == window_length:
        df['y'] = df['close'].shift(-i)
        df['y'] = df['y'] > df[f'close_{i-1}']
        df['datetime_id'] = df['datetime_id'].shift(-i)
    
    # Drop rows where there is a NaN
    df = df.dropna(axis=0)
        
    return df

In [7]:
window_size = 50
seq_df = window_input(window_size, data)

<ipython-input-5-1e7872469685>:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_{i}'] = df[col].shift(-i)
<ipython-input-5-1e7872469685>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df['y'] = df['close'].shift(-i)


In [9]:
X = seq_df.drop(['datetime_id', 'y'], axis=1)
y = seq_df['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)

In [13]:
from sklearn.naive_bayes import GaussianNB
model1 = GaussianNB()
model1.fit(X_train, y_train)
y_pred = model1.predict(X_test)
print(f"Accuracy {np.sum(y_pred == y_test) / len(y_test) * 100:.2f}%")

Accuracy 50.68%


In [15]:
from sklearn.ensemble import AdaBoostClassifier
model2 = AdaBoostClassifier(n_estimators=20, random_state=0)
model2.fit(X_train, y_train)
y_pred = model2.predict(X_test)
print(f"Accuracy {np.sum(y_pred == y_test) / len(y_test) * 100:.2f}%")

Accuracy 50.53%


In [17]:
from sklearn.neural_network import MLPClassifier
model3 = MLPClassifier(hidden_layer_sizes=100, random_state=0)
model3.fit(X_train, y_train)
y_pred = model3.predict(X_test)
print(f"Accuracy {np.sum(y_pred == y_test) / len(y_test) * 100:.2f}%")

Accuracy 49.41%


In [19]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
model4 = QuadraticDiscriminantAnalysis()
model4.fit(X_train, y_train)
y_pred = model4.predict(X_test)
print(f"Accuracy {np.sum(y_pred == y_test) / len(y_test) * 100:.2f}%")

Accuracy 50.94%


In [23]:
from sklearn.tree import DecisionTreeClassifier
model5 = DecisionTreeClassifier(max_depth=20)
model5.fit(X_train, y_train)
y_pred = model5.predict(X_test)
print(f"Accuracy {np.sum(y_pred == y_test) / len(y_test) * 100:.2f}%")

Accuracy 50.01%
